In [1]:
import pandas as pd

In [2]:
#df = pd.read_csv('CustomerSegmentation.csv' , index_col = 0 , encoding = 'Unicode_escape' )

In [3]:
!pip install fastai

In [4]:
 pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from fastai.collab import *
from fastai.tabular.all import *

In [ ]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine

In [ ]:
dbname = "PSweb"
user = "postgres"
password = "MANU19"
host = "localhost"
port = "5432"

In [ ]:
engine2 = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [ ]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)


In [ ]:

cur = conn.cursor()

In [ ]:
table_name = "customerordernew"
query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(table_name))

In [ ]:
# Execute the query
cur.execute(query)

In [ ]:
# Fetch all rows
rows = cur.fetchall()

In [ ]:
# Create a DataFrame from the fetched data
columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=columns)

In [ ]:
df.describe()

In [ ]:
df.info()



In [ ]:
df = df.drop(['description'], axis=1   )

In [ ]:
df['quantity'] = df['quantity'].astype(int)

In [ ]:
df = df.dropna()

In [ ]:
quant = df.groupby(['customerid', 'stockcode'])['quantity'].sum().reset_index()
quant = quant.rename(columns={'quantity': 'totalquantity'})

In [ ]:
quant.head()

In [ ]:
import math

In [ ]:
quant['totalquantity'] = quant['totalquantity'].astype(int)


In [ ]:
quant['totalquantity'].max()

In [ ]:
for i in range(len(quant)) :
   quant.loc[i, 'totalquantity'] = abs(quant['totalquantity'].iloc[i])


In [ ]:
dfm = quant 

In [ ]:
dfm.head()

In [ ]:
dfm.info()

In [ ]:
dls = CollabDataLoaders.from_df(dfm, item_name='stockcode', user_name='customerid', rating_name='totalquantity' , bs=64 , valid_pct= 0.2)
dls.show_batch()

In [ ]:
learner_inf = collab_learner(dls, y_range=(0, 100), layers=[100,50])
learner_inf.fit_one_cycle(8)

In [ ]:
learner_inf.model

In [ ]:
user_bias = learner_inf.model.i_bias.weight
item_bias = learner_inf.model.u_bias.weight
#idxs = item_bias.argsort(descending=True)[:5]
#[dls.classes['StockCode'][i] for i in idxs]

In [ ]:
#print(idxs)  #these are movie bias

In [ ]:
sale = learner_inf.model.i_weight.weight.squeeze()

In [ ]:
print(sale)

In [ ]:
user = learner_inf.model.u_weight.weight.squeeze()

In [ ]:
print(user)

In [ ]:
sale_trans = sale.t()

In [ ]:
mult = torch.matmul( user , sale_trans) 

In [ ]:
mult.size()

In [ ]:
adding = add(mult , item_bias)

In [ ]:
addingtra = adding.t()

In [ ]:
adding2 = add(addingtra , user_bias)

In [ ]:
final = adding2.t()

In [ ]:
final.size()

In [ ]:
print(final)

In [ ]:
out, inds = torch.max(final,dim=1) #since dim =1 ; we will get max of each row

In [ ]:
print(inds)

In [ ]:
dfd = pd.DataFrame()

In [ ]:
dfd['bestsale'] = 'yh'

In [ ]:
dfd['customerid'] = '0'

In [ ]:
dfd.head()

In [ ]:
for i in range(len(inds)) :
  dfd['bestsale'].loc[i] = dfm['stockcode'].iloc[int(inds[i])]

In [ ]:
dfd1 = dfm['customerid'].unique()

In [ ]:
print(dfd1)

In [ ]:
for i in range(len(dfd1)) :
    dfd['customerid'].loc[i] = dfd1[i]

In [ ]:
dff = pd.DataFrame(columns=['customerid'])

In [ ]:
dff = dff.rename(columns={'customerid': 'customerid'})

In [ ]:
for row in dfd1:
    dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)

In [ ]:
dff.head()

In [ ]:
dff.describe()

In [ ]:
dff1 = pd.DataFrame(columns=['sale'])

In [ ]:
arr = np.empty(len(inds), dtype=object)

In [ ]:
for i in range(len(inds)):
     arr[i] = dfm['stockcode'].iloc[int(inds[i])]

In [ ]:
#print(arr)

In [ ]:
#print(len(arr))

In [ ]:
dff1 = pd.DataFrame({'stockcode': arr})


In [ ]:
#ff1.head()

In [ ]:
#dff1.describe()

In [ ]:
result = pd.concat([dff, dff1], axis=1 )

In [ ]:
result.head()

In [ ]:
# Get the indices of the second-largest elements along each row
index2, indices_second_largest = torch.topk(final, k=2, dim=1)

# Use the indices to get the second-largest elements
second_largest_elements = torch.gather(final, dim=1, index=indices_second_largest[:, 1].view(-1, 1))

In [ ]:

# Convert the PyTorch tensor to a NumPy array
indices_second_largest_np = indices_second_largest.detach().numpy()
second_largest_indices = indices_second_largest_np[:, 1]


In [ ]:
print(second_largest_indices)

In [ ]:

# Convert the PyTorch tensor to a NumPy array
second_largest_elements_np = second_largest_elements.detach().numpy()

# Create a new DataFrame or add a new column to the existing DataFrame
#result['2best'] = second_largest_elements_np.tolist()

In [ ]:
result['bestsale2nd'] = result['customerid'] + 5

In [ ]:
for i in range(len(inds)) :
  result['bestsale2nd'].loc[i] = dfm['stockcode'].iloc[int(second_largest_indices[i])]

In [ ]:
result.head()

In [ ]:
##THIRD LARGEST INDEXX

In [ ]:
# Get the indices of the second-largest elements along each row
index3, indices_third_largest = torch.topk(final, k=3, dim=1)

In [ ]:

# Convert the PyTorch tensor to a NumPy array
indices_third_largest_np = indices_third_largest.detach().numpy()
third_largest_indices = indices_third_largest_np[:, 2]

In [ ]:
print(third_largest_indices)

In [ ]:
result['bestsale3rd'] = result['customerid'] + 5

In [ ]:
for i in range(len(inds)) :
  result['bestsale3rd'].loc[i] = dfm['stockcode'].iloc[int(third_largest_indices[i])]

In [ ]:
result.head()

In [ ]:
## 4 best sale 

In [ ]:
# Get the indices of the second-largest elements along each row
index4, indices_fourth_largest = torch.topk(final, k=5, dim=1)

In [ ]:
# Convert the PyTorch tensor to a NumPy array
indices_fourth_largest_np = indices_fourth_largest.detach().numpy()
fourth_largest_indices = indices_fourth_largest_np[:, 3]

In [ ]:
result['bestsale4th'] = result['customerid'] + 5

In [ ]:
for i in range(len(inds)) :
  result['bestsale4th'].loc[i] = dfm['stockcode'].iloc[int(fourth_largest_indices[i])]

In [ ]:
fifth_largest_indices = indices_fourth_largest_np[:, 4]

In [ ]:
result['bestsale5th'] = result['customerid'] + 5

In [ ]:
for i in range(len(inds)) :
  result['bestsale5th'].loc[i] = dfm['stockcode'].iloc[int(fifth_largest_indices[i])]

In [ ]:
result.head()

In [ ]:
result = result.dropna()

In [ ]:
table_name = 'customerrecostock'

In [ ]:
with engine2.connect() as connection:
    connection.execute(f"DELETE FROM {table_name}")

In [ ]:
result.to_sql(table_name, engine2, index=False, if_exists='append')

In [ ]:
#result.to_csv('finaloutput_recommender2.csv')